In [20]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
import re
import unidecode
from tqdm import tqdm
tqdm.pandas()

pd.options.display.max_columns=None

In [58]:
players = pd.read_csv('data/players.csv')
players.DoB = pd.to_datetime(players.DoB)

In [36]:
games = pd.read_csv('data/games.csv')

In [47]:
clubs = players.groupby('club_id').count().reset_index().loc[:,['club_id','club']]
clubs.to_csv('data/clubs.csv', index = False)

In [70]:
fifa22 = pd.read_csv('../../footprophet/Padda/data/Fifa/players_22.csv', usecols = ['sofifa_id','short_name','long_name','dob'])

In [71]:
fifa22.dob = pd.to_datetime(fifa22.dob)

In [232]:
players_bis = players.copy()

players_bis.name = players_bis.name.apply(unidecode.unidecode)
players_bis.short_name = players_bis.short_name.apply(unidecode.unidecode)

fifa22.long_name = fifa22.long_name.apply(unidecode.unidecode)
fifa22.short_name = fifa22.short_name.apply(unidecode.unidecode)



In [234]:
def search_name(x,fifa_N):
    fuzz_scores_ln = fifa_N.long_name.apply(lambda y: fuzz.token_set_ratio(x,y)).values   
    #each name in the "fifa" dataframe is assigned a similarity score with the selected name from the "players" dataset
    fuzz_scores_sn = fifa_N.short_name.apply(lambda y: fuzz.token_set_ratio(x,y)).values

    return np.argwhere(fuzz_scores_ln == fuzz_scores_ln.max()).squeeze() , fuzz_scores_ln.max(),\
        np.argwhere(fuzz_scores_sn == fuzz_scores_sn.max()).squeeze() , fuzz_scores_sn.max()

In [236]:
def search_all_names(fifa_N):
    results = players_bis.name.progress_apply(lambda x:search_name(x,fifa_N))
    
    players_bis['fifa_index_ln']= results.apply(lambda x : x[0])
    players_bis['fuzz_score_ln'] = results.apply(lambda x : x[1])
    players_bis['fifa_index_sn']= results.apply(lambda x : x[2])
    players_bis['fuzz_score_sn'] = results.apply(lambda x : x[3])
    
    return players_bis.loc[players_bis.fuzz_score_ln > 90,'fuzz_score_ln'].shape

In [237]:
search_all_names(fifa22)

100%|██████████| 715/715 [10:17<00:00,  1.16it/s]


(547,)

### long names don't match but short names match

In [238]:
players1 = players_bis.loc[(players.fuzz_score_ln < 80) & (players.fuzz_score_sn > 80),:].copy()
players1 = players1.drop(columns = ['fifa_index_ln','fuzz_score_ln']).reset_index(drop=True)

In [239]:
players1['name_compare'] = players1.fifa_index_sn.progress_apply(lambda x: [fifa22.loc[y,'long_name'] for y in x] if type(x)==np.ndarray else fifa22.loc[x,'long_name'])
players1['dob_compare'] = players1.fifa_index_sn.progress_apply(lambda x: [fifa22.loc[y,'dob'] for y in x] if type(x)==np.ndarray else fifa22.loc[x,'dob'])

100%|██████████| 71/71 [00:00<00:00, 12867.08it/s]


In [316]:
players1.shape

(0, 12)

In [ ]:
print(players1.loc[0,'lequipe_id'])
print(players1.loc[0,'name'])
print(players1.loc[0,'name_compare'])
print(players1.loc[0,'DoB'])
print(players1.loc[0,'dob_compare'])
print(players1.loc[0,'fifa_index_sn'])

players1 = players1.drop(0).reset_index(drop=True)


Da Silva Fabio (29842) => 3180  
Eiji Kawashima (36275) => 2476   
Habib Diallo (54331) => 1779  
Marquinhos (46961) => 39  
Metinho (87695) => 15050  
Hyun-jun Suk (35942) => 6539  
Pau Lopez (53546) => 462  
Alvaro Gonzalez (41014) => 562  
Tiago Djalo (76323) => 2412
Vital Nsimba (42262) => 4283  
Capita (83797) => 14163  
Bruno Guimaraes (68534) => 659  
Lucas Paqueta (70628) => 374  
Lucien Agoume (70963) => 10706
Xeka (61723) => 1089  
Tino Kadewere (69206) => 1050  
Edson Mexer (35518) => 3224  
Fransergio (52475) => 713  
Otavio (58019) => 1327


### long name match but multiple fifa_index

In [332]:
players1 = players_bis.loc[(players_bis.fifa_index_ln.apply(type) != int) & (players_bis.fuzz_score_ln >=80) , : ].reset_index(drop=True).copy()

In [333]:
players1['name_compare'] = players1.fifa_index_ln.progress_apply(lambda x: [fifa22.loc[y,'long_name'] for y in x] if type(x)==np.ndarray else fifa22.loc[x,'long_name'])
players1['dob_compare'] = players1.fifa_index_ln.progress_apply(lambda x: [fifa22.loc[y,'dob'] for y in x] if type(x)==np.ndarray else fifa22.loc[x,'dob'])

100%|██████████| 20/20 [00:00<00:00, 5434.80it/s]


In [334]:
players1.shape

(20, 14)

In [ ]:

print(players1.loc[0,'lequipe_id'])
print(players1.loc[0,'name'])
print(players1.loc[0,'name_compare'])
print(players1.loc[0,'DoB'])
print(players1.loc[0,'dob_compare'])
print(players1.loc[0,'fifa_index_ln'])

players1 = players1.drop(0).reset_index(drop=True)

Mohamed-Ali Cho (82985) => 9686  
Nuno Mendes (76513) => 861  
Dante (20255) => 1147  
Danilo Pereira (41511) => 317  
Gabriel Silva (44649) => 3229  
Justin Smith (91006) => 16495  
Sambou Sissoko (64659) => 16061  
Emerson (55093) => 751  
Gerson (60412) => 483  
Pape Gueye (63063) => 1428  
Igor Silva(57877) => 7192  
Henrique (73302) => 4153  
Boubacar Traore (77098) => 14831  
Jonathan David (68269) => 848   
Ibrahima Balde (83030) => 17379  
Gideon Mensah (65406) => 4525  
Otavio (58019) => 1327


### long names that almost match (>80 < 95)

In [358]:
players1 = players_bis.loc[(players_bis.fuzz_score_ln >= 80) & (players_bis.fuzz_score_ln <=95) & (players_bis.fifa_index_ln.apply(type) == int), : ].reset_index(drop=True).copy()

In [359]:
players1['name_compare'] = players1.fifa_index_ln.progress_apply(lambda x: [fifa22.loc[y,'long_name'] for y in x] if type(x)==np.ndarray else fifa22.loc[x,'long_name'])
players1['dob_compare'] = players1.fifa_index_ln.progress_apply(lambda x: [fifa22.loc[y,'dob'] for y in x] if type(x)==np.ndarray else fifa22.loc[x,'dob'])

100%|██████████| 27/27 [00:00<00:00, 11905.61it/s]


In [ ]:
print(players1.loc[0,'lequipe_id'])
print(players1.loc[0,'name'])
print(players1.loc[0,'name_compare'])
print(players1.loc[0,'DoB'])
print(players1.loc[0,'dob_compare'])
print(players1.loc[0,'fifa_index_ln'])

players1 = players1.drop(0).reset_index(drop=True)

Alexandre Golovine (56524)  => 616  
Boubacar Kouyate (71973) => 3049  
Papa Ndiaga Yade (76413) => 8471  
Hianga'a Mbock (75905) => 10789


### correction du df players

Da Silva Fabio (29842) => 3180  
Eiji Kawashima (36275) => 2476   
Habib Diallo (54331) => 1779  
Marquinhos (46961) => 39  
Metinho (87695) => 15050  
Hyun-jun Suk (35942) => 6539  
Pau Lopez (53546) => 462  
Alvaro Gonzalez (41014) => 562  
Tiago Djalo (76323) => 2412
Vital Nsimba (42262) => 4283  
Capita (83797) => 14163  
Bruno Guimaraes (68534) => 659  
Lucas Paqueta (70628) => 374  
Lucien Agoume (70963) => 10706
Xeka (61723) => 1089  
Tino Kadewere (69206) => 1050  
Edson Mexer (35518) => 3224  
Fransergio (52475) => 713  
Otavio (58019) => 1327

Mohamed-Ali Cho (82985) => 9686  
Nuno Mendes (76513) => 861  
Dante (20255) => 1147  
Danilo Pereira (41511) => 317  
Gabriel Silva (44649) => 3229  
Justin Smith (91006) => 16495  
Sambou Sissoko (64659) => 16061  
Emerson (55093) => 751  
Gerson (60412) => 483  
Pape Gueye (63063) => 1428  
Igor Silva(57877) => 7192  
Henrique (73302) => 4153  
Boubacar Traore (77098) => 14831  
Jonathan David (68269) => 848   
Ibrahima Balde (83030) => 17379  
Gideon Mensah (65406) => 4525  
Otavio (58019) => 1327

Alexandre Golovine (56524)  => 616  
Boubacar Kouyate (71973) => 3049  
Papa Ndiaga Yade (76413) => 8471  
Hianga'a Mbock (75905) => 10789

39

In [412]:
player_change_list = [29842, 36275, 54331, 46961, 87695, 35942, 53546, 41014, 76323, 42262, 83797, 68534, 70628, 70963, 61723, 69206, 35518, 52475, 82985, \
    76513, 20255, 41511, 44649, 91006, 64659, 55093, 60412, 63063, 57877, 73302, 77098, 68269, 83030, 65406, 58019, 56524, 71973, 76413, 75905]
fifa_index_list = [3180,2476,1779,39,15050,6539,462,562,2412,4283,14163,659,374,10706,1089,1050,3224,713,9686,861,1147,317,3229,16495,16061,751,483,\
    1428,7192,4153,14831,848,17379,4525,1327,616,3049,8471,10789]
len(player_change_list), len(fifa_index_list)

(39, 39)

In [420]:
players_bis.head(1)

,club,club_id,number,name,short_name,DoB,poste,lequipe_id,fifa_index_ln,fuzz_score_ln,fifa_index_sn,fuzz_score_sn
0,Angers,374,16.0,Anthony Mandrea,A. Mandrea,1996-12-25,Gar.,50523,12485,100,"[11751, 12485]",88


In [422]:
for i, player_id in enumerate(player_change_list):
    players_bis.loc[players_bis.lequipe_id == player_id,'fuzz_score_ln'] = 100
    players_bis.loc[players_bis.lequipe_id == player_id,'fifa_index_ln'] = fifa_index_list[i]

In [442]:
#players_bis.loc[(players_bis.fuzz_score_ln < 95) & (~players_bis.number.isnull()),:]

In [438]:
display(players_bis.loc[players_bis.lequipe_id==40096,:])
players_bis.loc[players_bis.lequipe_id==40096,'fuzz_score_ln'] = 100
players_bis.loc[players_bis.lequipe_id==40096,'fifa_index_ln'] = 3215

,club,club_id,number,name,short_name,DoB,poste,lequipe_id,fifa_index_ln,fuzz_score_ln,fifa_index_sn,fuzz_score_sn
432,Monaco,25,28.0,Judilson Pele,J. Pele,1991-09-29,Mil.,40096,3215,100,3215,100


In [436]:
players_bis.loc[players_bis.lequipe_id ==69885,'fuzz_score_ln']=100

In [441]:
fifa22.loc[fifa22.sofifa_id==161648,:]

,sofifa_id,short_name,long_name,dob


# new entries

bradley barcola (83812) => sofifa :264652  
Loum Tchaouna (88826) => 264880  
Kayne Bonnevie (72364) => 264871  
Yanis Lhery (87893) => 265687  
Felix Lemarechal (90544) => 264869  
Jean-Armel Kana-Biyik (28316) => 183975  
Theo Le Bris (77183) => 265544  
Louis Mafouta (58777)=> 266586  
Yadaly Diaby (93026) => 265689  
Paul Lasne (255000000000000000000025947)=> 192242  
Hatem Ben Arfa (20671) => 161648  
Samba Sow (18955) => 191151  
Nabil Bentaleb (50329) => 213296  
Maghnes Akliouche (83186)=> 264862  
Dango Ouattara (87013)=> 265552

In [457]:
player_change_list = [83812, 88826, 72364, 87893, 90544, 28316, 77183, 58777, 93026, 255000000000000000000025947, 20671, 18955, 50329, 83186, 87013]
sofifa_id_list =[
    264652, 264880, 264871, 265687, 264869, 183975, 265544, 266586, 265689, 192242, 161648, 191151, 213296, 264862, 265552
]
len(player_change_list), len(fifa_index_list)

(15, 15)

In [446]:
for i, player_id in enumerate(player_change_list):
    players_bis.loc[players_bis.lequipe_id == player_id,'fuzz_score_ln'] = 100
    players_bis.loc[players_bis.lequipe_id == player_id,'fifa_index_ln'] = None

In [450]:
players_good = players_bis.loc[players_bis.fuzz_score_ln>=95,['club','club_id','number','name','short_name','DoB','poste','lequipe_id','fifa_index_ln']].reset_index(drop=True).copy()

In [455]:
print(f'il manque {players_bis.shape[0] - players_good.shape[0]} joueurs sur {players_bis.shape[0]}')

il manque 138 joueurs sur 715


In [458]:
players_good['sofifa_id'] = None
players_good.loc[players_good.fifa_index_ln.isnull(),:]

,club,club_id,number,name,short_name,DoB,poste,lequipe_id,fifa_index_ln,sofifa_id
10,Angers,374,23.0,Nabil Bentaleb,N. Bentaleb,1994-11-24,Mil.,50329,None,None
197,Rennes,14,17.0,Loum Tchaouna,L. Tchaouna,2003-08-09,Att.,88826,None,None
291,Monaco,25,35.0,Maghnes Akliouche,M. Akliouche,2002-02-25,Mil.,83186,None,None
323,Lyon (OL),22,40.0,Kayne Bonnevie,K. Bonnevie,NaT,Gar.,72364,None,None
332,Saint-Étienne (ASSE),38,34.0,Yanis Lhery,Y. Lhery,2003-07-05,Att.,87893,None,None
342,Monaco,25,41.0,Felix Lemarechal,F. Lemarechal,2003-07-08,Mil.,90544,None,None
346,Metz,20,27.0,Jean-Armel Kana-Biyik,J. Kana-Biyik,1989-03-07,Déf.,28316,None,None
376,Lorient,11,37.0,Theo Le Bris,T. Le Bris,2002-01-10,Mil.,77183,None,None
428,Metz,20,34.0,Louis Mafouta,L. Mafouta,1994-02-07,Att.,58777,None,None
455,Lorient,11,38.0,Dango Ouattara,D. Ouattara,2002-11-02,Att.,87013,None,None


In [459]:
for i, player_id in enumerate(player_change_list):
    players_good.loc[players_good.lequipe_id==player_id, 'sofifa_id'] = sofifa_id_list[i]
    
players_good.loc[players_good.fifa_index_ln.isnull(),:]

,club,club_id,number,name,short_name,DoB,poste,lequipe_id,fifa_index_ln,sofifa_id
10,Angers,374,23.0,Nabil Bentaleb,N. Bentaleb,1994-11-24,Mil.,50329,None,213296
197,Rennes,14,17.0,Loum Tchaouna,L. Tchaouna,2003-08-09,Att.,88826,None,264880
291,Monaco,25,35.0,Maghnes Akliouche,M. Akliouche,2002-02-25,Mil.,83186,None,264862
323,Lyon (OL),22,40.0,Kayne Bonnevie,K. Bonnevie,NaT,Gar.,72364,None,264871
332,Saint-Étienne (ASSE),38,34.0,Yanis Lhery,Y. Lhery,2003-07-05,Att.,87893,None,265687
342,Monaco,25,41.0,Felix Lemarechal,F. Lemarechal,2003-07-08,Mil.,90544,None,264869
346,Metz,20,27.0,Jean-Armel Kana-Biyik,J. Kana-Biyik,1989-03-07,Déf.,28316,None,183975
376,Lorient,11,37.0,Theo Le Bris,T. Le Bris,2002-01-10,Mil.,77183,None,265544
428,Metz,20,34.0,Louis Mafouta,L. Mafouta,1994-02-07,Att.,58777,None,266586
455,Lorient,11,38.0,Dango Ouattara,D. Ouattara,2002-11-02,Att.,87013,None,265552


In [467]:
players_good.head(1)

,club,club_id,number,name,short_name,DoB,poste,lequipe_id,fifa_index_ln,sofifa_id
0,Angers,374,16.0,Anthony Mandrea,A. Mandrea,1996-12-25,Gar.,50523,12485,220833


In [464]:
fifa22.loc[12485,'sofifa_id']

220833

In [466]:
players_good.loc[~players_good.fifa_index_ln.isnull(),'sofifa_id'] = \
    players_good.loc[~players_good.fifa_index_ln.isnull(),'fifa_index_ln'].apply(lambda x: fifa22.loc[x,'sofifa_id'])

In [478]:
players = pd.merge(players,players_good.loc[:,['lequipe_id','sofifa_id']], on = ['lequipe_id'], how = 'left')

In [480]:
players.to_csv('players_sofifa.csv', index=False)